In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio

In [2]:
pio.templates.default = 'plotly_white'
px.set_mapbox_access_token(open(".mapbox_token").read())
PALETTE = px.colors.qualitative.Prism
MAIN_COLOR = 'rgb(231,41,138)'

In [3]:
def create_df(file_name, cols, na_cols=None):
  df = pd.read_csv(file_name).drop(cols, axis=1)
  df[na_cols] = df[na_cols].fillna('No info')
  return df

In [4]:
cols = ['license', 'neighbourhood_group']
na_cols = ['name']
df_listing = create_df('data/listings_short.csv', cols, na_cols)

In [6]:
df_listing.isna().sum().sort_values(ascending=False)

last_review                       702
reviews_per_month                 702
id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
dtype: int64

In [30]:
df_reviews = pd.read_csv('data/reviews_2023_03.csv', parse_dates=['date'])
df_final = df_reviews.merge(df_listing, left_on='listing_id', right_on='id', how='inner').drop(['id_x', 'id_y'], axis=1)

In [8]:
len(df_final)

93656

In [9]:
df_final.isna().sum()

listing_id                        0
date                              0
reviewer_id                       0
reviewer_name                     0
comments                          2
name                              0
host_id                           0
host_name                         0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
number_of_reviews_ltm             0
dtype: int64

In [10]:
df_final[df_final['comments'].isna() == True]

,listing_id,date,reviewer_id,reviewer_name,comments,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
47701,25051575,2019-04-23,244070000,David,NaN,Stor villa med sol hela dagen,26990407,Moa,Skarpnäcks,59.27726,18.11034,Entire home/apt,1990,5,24,2023-01-03,0.42,1,27,7
62929,31301631,2019-05-30,263373046,Deepak,NaN,Superior Studio Apartment with Sofa Bed,362541894,ApartDirect,Älvsjö,59.28055,18.01409,Entire home/apt,836,2,83,2023-03-13,1.63,17,266,24


In [6]:
df_final.dropna(subset=['comments'], inplace=True)

In [12]:
len(df_final)

93654

In [13]:
df_final.isna().sum()

listing_id                        0
date                              0
reviewer_id                       0
reviewer_name                     0
comments                          0
name                              0
host_id                           0
host_name                         0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
number_of_reviews_ltm             0
dtype: int64

In [14]:
df_final['reviewer_id'].value_counts()

348992094    31
24622700     25
110641325    16
10805888     11
280949885    11
             ..
32023642      1
59702442      1
38577770      1
42229921      1
77261435      1
Name: reviewer_id, Length: 88204, dtype: int64

In [15]:
df_final.query('reviewer_id == 348992094').head(3)

,listing_id,date,reviewer_id,reviewer_name,comments,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
53092,31347264,2020-11-30,348992094,Roger,Bra läge och mycket smidig in och utcheckning,# 12 Studiolägenhet utan fönster(no window),8098890,Robin,Kungsholmens,59.33435,18.02585,Entire home/apt,621,2,224,2023-03-30,4.39,20,332,58
53097,31347264,2021-02-12,348992094,Roger,Bra läge och alltid bra kommunikation. Allt fu...,# 12 Studiolägenhet utan fönster(no window),8098890,Robin,Kungsholmens,59.33435,18.02585,Entire home/apt,621,2,224,2023-03-30,4.39,20,332,58
53099,31347264,2021-03-20,348992094,Roger,Bra läge och mycket bra boende,# 12 Studiolägenhet utan fönster(no window),8098890,Robin,Kungsholmens,59.33435,18.02585,Entire home/apt,621,2,224,2023-03-30,4.39,20,332,58


In [7]:
le = LabelEncoder()
df_final['listing_id_encod'] = le.fit_transform(df_final['listing_id'])

In [19]:
px.histogram(df_final, x='listing_id_encod')

In [8]:
num_reviews = (df_final.groupby(['listing_id_encod', 'neighbourhood'])
          .size().sort_values(ascending=False)
          .reset_index()
          .rename(columns={0: 'num_reviews'}))
num_reviews

,listing_id_encod,neighbourhood,num_reviews
0,1073,Södermalms,976
1,27,Östermalms,675
2,1329,Norrmalms,653
3,7,Södermalms,578
4,545,Södermalms,515
...,...,...,...
2793,2180,Norrmalms,1
2794,2177,Östermalms,1
2795,2170,Älvsjö,1
2796,2161,Södermalms,1


In [9]:
px.bar(num_reviews.nlargest(50, 'num_reviews'), y='num_reviews', 
       color='neighbourhood', color_discrete_sequence=PALETTE)

In [10]:
fig = px.bar(num_reviews.groupby('neighbourhood')['num_reviews'].sum().sort_values(ascending=True), orientation='h')
fig.update_traces(marker_color=MAIN_COLOR)

In [24]:
df_listing.dropna(inplace=True)

In [26]:
len(df_listing)

2798

In [25]:
fig = ff.create_hexbin_mapbox(
    data_frame=df_listing, lat="latitude", lon="longitude",
    nx_hexagon=10, opacity=0.9, labels={"color": "Point Count"},
    color_continuous_scale=PALETTE,
    #show_original_data=True
)
fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
fig.show()

In [27]:
df_listing['price'].agg(['max', 'min', 'mean'])

max     480500.000000
min          0.000000
mean      1697.782702
Name: price, dtype: float64

In [28]:
df_listing.query('price > 20000')

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
755,19529551,Sunny apartment 27 mins to Centralen,137155339,Hillerodsgrand,Rinkeby-Tensta,59.395560,17.949290,Entire home/apt,480500,7,8,2019-08-01,0.12,1,29,0
2202,53198829,2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.373330,17.837490,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5
2270,54126655,Small privately owned hotel 12 rooms and 22 beds,280607063,Hotel Söder,Södermalms,59.315224,18.077539,Private room,22995,1,1,2022-12-30,0.33,7,364,1


In [33]:
df_final.query('listing_id == 53198829')

,listing_id,date,reviewer_id,reviewer_name,comments,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
83874,53198829,2022-01-07,235853369,Amygdalia,Camilla was an excellent host! She was kind an...,2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.37333,17.83749,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5
83875,53198829,2022-01-25,59681973,Kassem,Highly recommended,2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.37333,17.83749,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5
83876,53198829,2022-01-28,59681973,Kassem,"Camilla is a very good host, very responsive a...",2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.37333,17.83749,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5
83877,53198829,2022-02-04,127334280,Chaudhary,Camilla is an excellent host. Very sweet and c...,2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.37333,17.83749,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5
83878,53198829,2022-02-20,342541532,Resha,"Kan stark rekommendera lägenheten, precis som ...",2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.37333,17.83749,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5
83879,53198829,2022-03-21,443651639,Habid,Super cool! Definitely one of the best Airbnb’...,2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.37333,17.83749,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5
83880,53198829,2022-04-16,422594941,Gloria,Camilla’s flat is really beautiful and equippe...,2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.37333,17.83749,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5
83881,53198829,2022-05-02,235853369,Amygdalia,Camilla är bäst! Jag rekommenderar henne stark...,2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.37333,17.83749,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5
83882,53198829,2022-05-29,448748591,Peter,"Bästa kvalitet inom alla områden, kommunikatio...",2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.37333,17.83749,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5
83883,53198829,2022-06-06,123730673,Chelsea,Camilla’s place was great. The rooms were clea...,2:a i Hässelby,430664934,Camilla,Hässelby-Vällingby,59.37333,17.83749,Entire home/apt,450590,2,11,2022-12-11,0.73,1,1,5


In [46]:
df_check_listing = pd.read_csv('data/listings.csv')
df_check_listing['price'] = df_check_listing['price'].str.replace(',', '').str.replace('$', '').astype(float)

/var/folders/_r/g64kt4sn647_vn13t1kvjhk80000gp/T/ipykernel_25278/2964605359.py:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [50]:
df_check_listing[['id', 'price']].query('price > 20000')

,id,price
372,8406721,672896.0
754,19529551,411926.0
2483,599691967730847088,22995.0
2484,599723488406885421,22995.0
2527,616448454134473600,500550.0
2709,651125297966010002,73033.0
2783,662708957070867730,30000.0
2859,675152069709618971,22995.0
3274,776549730665441181,99999.0


In [21]:
px.histogram(df_listing, x='price')

In [12]:
fig = px.scatter_mapbox(df_listing, 
                            lat="latitude", lon="longitude", 
                            color="price", size="number_of_reviews",
                            color_continuous_scale=PALETTE,
                            size_max=15, zoom=10)
fig.show()